In [1]:
import prophet as pro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_sitting = pd.read_csv('resources/success_trans0725_0824.csv', parse_dates=['time'])
df_standing = pd.read_csv('resources/failed_trans0725_0824.csv', parse_dates=['time'])

In [3]:
df_sitting.shape

(8641, 2)

In [4]:
df_standing.shape

(5040, 2)

In [5]:
df_sitting.head()

,time,sitting_people
0,2023-07-25 18:00:00+00:00,814
1,2023-07-25 18:05:00+00:00,954
2,2023-07-25 18:10:00+00:00,1054
3,2023-07-25 18:15:00+00:00,1274
4,2023-07-25 18:20:00+00:00,1766


In [7]:
df_sitting.tail()

,time,sitting_people
8636,2023-08-24 18:20:00+00:00,1674
8637,2023-08-24 18:25:00+00:00,1861
8638,2023-08-24 18:30:00+00:00,1561
8639,2023-08-24 18:35:00+00:00,1486
8640,2023-08-24 18:40:00+00:00,1579


In [8]:
df_standing.head()

,time,standing_people
0,2023-07-25 19:00:00+00:00,12
1,2023-07-25 19:05:00+00:00,14
2,2023-07-25 19:10:00+00:00,26
3,2023-07-25 19:15:00+00:00,16
4,2023-07-25 19:20:00+00:00,14


In [9]:
df_standing.tail()

,time,standing_people
5035,2023-08-24 18:35:00+00:00,3
5036,2023-08-24 18:40:00+00:00,2
5037,2023-08-24 18:45:00+00:00,2
5038,2023-08-24 18:50:00+00:00,5
5039,2023-08-24 19:00:00+00:00,11


There seems to be some disparage between df_sitting(8641) and df_standing (5040).

In [10]:
# Taking exactly 30 days of sitting_transactions with 5 min interval
df_sitting = df_sitting[:8632]

In [11]:
df_sitting.shape

(8632, 2)

In [12]:
df_sitting.tail()

,time,sitting_people
8627,2023-08-24 17:35:00+00:00,1925
8628,2023-08-24 17:40:00+00:00,1747
8629,2023-08-24 17:45:00+00:00,1825
8630,2023-08-24 17:50:00+00:00,1734
8631,2023-08-24 17:55:00+00:00,2037


In [13]:
df_sitting.head()

,time,sitting_people
0,2023-07-25 18:00:00+00:00,814
1,2023-07-25 18:05:00+00:00,954
2,2023-07-25 18:10:00+00:00,1054
3,2023-07-25 18:15:00+00:00,1274
4,2023-07-25 18:20:00+00:00,1766


In [14]:
df_sitting.columns = ['DS', 'Y']

In [15]:
df_sitting.head()

,DS,Y
0,2023-07-25 18:00:00+00:00,814
1,2023-07-25 18:05:00+00:00,954
2,2023-07-25 18:10:00+00:00,1054
3,2023-07-25 18:15:00+00:00,1274
4,2023-07-25 18:20:00+00:00,1766


In [19]:
df_sitting.dtypes

DS    datetime64[ns, UTC]
Y                   int64
dtype: object

With 5 min interval and 30 days of sitting people recorded there should be 288 * 30 = 8640 transactions. 

In [16]:
df_sitting.shape

(8632, 2)

Some of the transactions are missing. Let's find out those timings.

In [18]:
start_date = '2023-07-25 18:00:00'
end_date = '2023-08-24 17:55:00'

In [20]:
df_date = pd.date_range(start=start_date, end=end_date, freq='5T')

In [23]:
date_df = df_date.to_frame(index=False, name='DS')

In [24]:
date_df.head()

,DS
0,2023-07-25 18:00:00
1,2023-07-25 18:05:00
2,2023-07-25 18:10:00
3,2023-07-25 18:15:00
4,2023-07-25 18:20:00


In [25]:
df_sitting['DS'] = df_sitting['DS'].dt.tz_localize(None)

In [26]:
df_sitting.head()

,DS,Y
0,2023-07-25 18:00:00,814
1,2023-07-25 18:05:00,954
2,2023-07-25 18:10:00,1054
3,2023-07-25 18:15:00,1274
4,2023-07-25 18:20:00,1766


In [27]:
date_df.tail()

,DS
8635,2023-08-24 17:35:00
8636,2023-08-24 17:40:00
8637,2023-08-24 17:45:00
8638,2023-08-24 17:50:00
8639,2023-08-24 17:55:00


In [28]:
df_sitting.tail()

,DS,Y
8627,2023-08-24 17:35:00,1925
8628,2023-08-24 17:40:00,1747
8629,2023-08-24 17:45:00,1825
8630,2023-08-24 17:50:00,1734
8631,2023-08-24 17:55:00,2037


In [32]:
cleaned_df_sitting = date_df.merge(df_sitting, on='DS', how='left') 

In [33]:
cleaned_df_sitting.shape

(8640, 2)

In [35]:
cleaned_df_sitting[cleaned_df_sitting.isna().any(axis=1)]

,DS,Y
6004,2023-08-15 14:20:00,NaN
6005,2023-08-15 14:25:00,NaN
6006,2023-08-15 14:30:00,NaN
6007,2023-08-15 14:35:00,NaN
6008,2023-08-15 14:40:00,NaN
6009,2023-08-15 14:45:00,NaN
6010,2023-08-15 14:50:00,NaN
6011,2023-08-15 14:55:00,NaN


So we do have some NaN values and they are around 8 timestamps. So let's replace them with zeros (meaning that there are no sitting people)

In [36]:
cleaned_df_sitting.fillna(0, inplace=True)

In [37]:
cleaned_df_sitting[cleaned_df_sitting.isna().any(axis=1)]

,DS,Y


In [39]:
cleaned_df_sitting.to_csv('resources/preprocessed_sitting_trans.csv', index=False)

Now let's look into the df_standing dataframe.

In [40]:
df_standing.shape

(5040, 2)

In [41]:
df_standing.head()

,time,standing_people
0,2023-07-25 19:00:00+00:00,12
1,2023-07-25 19:05:00+00:00,14
2,2023-07-25 19:10:00+00:00,26
3,2023-07-25 19:15:00+00:00,16
4,2023-07-25 19:20:00+00:00,14


In [43]:
df_standing.columns = ['DS', 'Y']

In [44]:
df_standing.head()

,DS,Y
0,2023-07-25 19:00:00+00:00,12
1,2023-07-25 19:05:00+00:00,14
2,2023-07-25 19:10:00+00:00,26
3,2023-07-25 19:15:00+00:00,16
4,2023-07-25 19:20:00+00:00,14


In [45]:
df_standing['DS'] = df_standing['DS'].dt.tz_localize(None)

In [46]:
df_standing.head()

,DS,Y
0,2023-07-25 19:00:00,12
1,2023-07-25 19:05:00,14
2,2023-07-25 19:10:00,26
3,2023-07-25 19:15:00,16
4,2023-07-25 19:20:00,14


In [47]:
df_standing.tail()

,DS,Y
5035,2023-08-24 18:35:00,3
5036,2023-08-24 18:40:00,2
5037,2023-08-24 18:45:00,2
5038,2023-08-24 18:50:00,5
5039,2023-08-24 19:00:00,11


In [48]:
date_df.head()

,DS
0,2023-07-25 18:00:00
1,2023-07-25 18:05:00
2,2023-07-25 18:10:00
3,2023-07-25 18:15:00
4,2023-07-25 18:20:00


In [49]:
cleaned_standing_df = date_df.merge(df_standing, on='DS', how='left')

In [50]:
cleaned_standing_df.shape

(8640, 2)

In [51]:
len(cleaned_standing_df[cleaned_standing_df.isna().any(axis=1)])

3609

In [52]:
cleaned_standing_df.fillna(0, inplace=True)

In [53]:
len(cleaned_standing_df[cleaned_standing_df.isna().any(axis=1)])

0

In [54]:
cleaned_standing_df.to_csv('resources/preprocessed_standing_trans.csv', index=False )